# HoneyBee Clinical Processing - Complete Guide

This comprehensive notebook demonstrates all capabilities of the HoneyBee clinical processing system, including:

- 📄 **PDF Processing** with OCR
- 📝 **Text Processing** with entity extraction
- 🧬 **Cancer-Specific Entity Recognition**
- ⏰ **Temporal Timeline Extraction**
- 🤖 **Embedding Generation** with multiple biomedical models
- ⚙️ **Advanced Configuration** options
- 📦 **Batch Processing** capabilities
- 🔗 **HoneyBee API** integration

---

**Documentation:** https://lab-rasool.github.io/HoneyBee/docs/clinical-processing/

**Author:** HoneyBee Development Team  
**Version:** 1.0  
**Date:** October 2025

## 🚀 Setup and Installation

### System Dependencies
```bash
# Ubuntu/Debian
sudo apt-get install openslide-tools tesseract-ocr

# macOS
brew install openslide tesseract
```

### Python Dependencies
```bash
pip install torch transformers pytesseract pillow PyPDF2 pdf2image nltk opencv-python
```

### NLTK Data
```python
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
```

In [ ]:
# Import required libraries
import sys
import json
import warnings
import numpy as np
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Import HoneyBee components
from honeybee import HoneyBee
from honeybee.processors import ClinicalProcessor

print("✓ All imports successful!")
print(f"Working directory: {Path.cwd()}")

---
# 📄 Section 1: Basic PDF Processing

Process clinical documents (PDF, images) with OCR support.

In [ ]:
# Initialize Clinical Processor
processor = ClinicalProcessor()

# Path to sample PDF
pdf_path = Path("sample.PDF")

if pdf_path.exists():
    print(f"Processing: {pdf_path.name}")
    print("-" * 60)
    
    # Process the PDF
    result = processor.process(pdf_path, save_output=False)
    
    # Display results
    print(f"\n✓ PDF Processed Successfully")
    print(f"\nExtraction Details:")
    print(f"  Method: {result['document_processing']['method']}")
    print(f"  Text Length: {len(result['text'])} characters")
    print(f"  Entities Extracted: {len(result['entities'])}")
    print(f"  Document Sections: {result['document_structure']['num_sections']}")
    
    # Show first 500 characters
    print(f"\nExtracted Text (first 500 chars):")
    print("=" * 60)
    print(result['text'][:500] + "...")
    print("=" * 60)
    
    # Show extracted entities
    print(f"\nExtracted Entities (first 10):")
    for i, entity in enumerate(result['entities'][:10], 1):
        print(f"  {i}. {entity['text']:30s} [{entity['type']}]")
else:
    print(f"⚠ PDF not found: {pdf_path}")

---
# 📝 Section 2: Direct Text Processing

Process clinical text directly without files.

In [ ]:
# Sample clinical text
clinical_text = """
PATHOLOGY REPORT

Patient: Jane Smith
Date: March 15, 2024

DIAGNOSIS: Invasive Ductal Carcinoma, Right Breast

CLINICAL HISTORY:
Patient presented on February 20, 2024 with a palpable 2.5 cm mass in the right
upper outer quadrant. Core biopsy on February 28, 2024 confirmed invasive carcinoma.

MICROSCOPIC FINDINGS:
Invasive ductal carcinoma, Grade 2 (Nottingham score 6)
- Tubule formation: 20% (score 2)
- Nuclear pleomorphism: Moderate (score 2)
- Mitotic rate: 8 per 10 HPF (score 2)

IMMUNOHISTOCHEMISTRY:
ER: Positive (95%, strong)
PR: Positive (80%, moderate)
HER2: Negative (IHC 1+)
Ki-67: 18%

TNM: pT2 N0 M0, Stage IIA

TREATMENT:
Started tamoxifen 20 mg PO daily on March 25, 2024
Follow-up: May 1, 2024
"""

print("Processing Clinical Text...")
print("=" * 60)

# Process the text
result = processor.process_text(clinical_text, document_type="pathology_report")

print(f"\n✓ Text Processed Successfully")
print(f"\nResults:")
print(f"  Document Type: {result['document_type']}")
print(f"  Text Length: {len(result['text'])} characters")
print(f"  Entities: {len(result['entities'])}")
print(f"  Timeline Events: {len(result['temporal_timeline'])}")
print(f"  Entity Relationships: {len(result['entity_relationships'])}")

In [ ]:
# Display entity breakdown by type
entity_types = {}
for entity in result['entities']:
    et = entity['type']
    entity_types[et] = entity_types.get(et, 0) + 1

print("\nEntity Type Breakdown:")
print("-" * 40)
for entity_type in sorted(entity_types.keys()):
    count = entity_types[entity_type]
    bar = "█" * min(count, 30)
    print(f"{entity_type:20s} {count:3d} {bar}")

In [ ]:
# Display temporal timeline
if result['temporal_timeline']:
    print("\nTemporal Timeline:")
    print("=" * 60)
    
    for i, event in enumerate(result['temporal_timeline'], 1):
        print(f"\n{i}. {event['temporal_text']}")
        if event.get('normalized_date'):
            print(f"   Normalized: {event['normalized_date']}")
        
        related = event.get('related_entities', [])
        if related:
            print(f"   Related entities: {len(related)}")
            for idx in related[:3]:  # Show first 3
                if idx < len(result['entities']):
                    e = result['entities'][idx]
                    print(f"     • {e['text']} ({e['type']})")
else:
    print("\nNo timeline events found")

---
# ⚙️ Section 3: Configuration Options

Customize processing with different strategies and options.

### 3.1 Sentence Segmentation with Sliding Window

In [ ]:
# Configuration for sentence-based segmentation
config_sentence = {
    "tokenization": {
        "model": "gatortron",
        "segment_strategy": "sentence",
        "long_document_strategy": "sliding_window",
        "stride": 128
    },
    "entity_recognition": {
        "cancer_specific_extraction": True
    }
}

processor_sentence = ClinicalProcessor(config_sentence)

# Create a longer text for testing
long_text = clinical_text * 5

result_sentence = processor_sentence.process_text(long_text)

print("Sentence Segmentation + Sliding Window:")
print(f"  Text Length: {len(result_sentence['text'])} characters")
print(f"  Strategy: {result_sentence.get('tokenization', {}).get('tokenization_strategy', 'N/A')}")
if 'num_windows' in result_sentence.get('tokenization', {}):
    print(f"  Windows: {result_sentence['tokenization']['num_windows']}")
print(f"  Entities: {len(result_sentence['entities'])}")

### 3.2 Important Segments Strategy

In [ ]:
# Configuration for important segments
config_important = {
    "tokenization": {
        "model": "gatortron",
        "long_document_strategy": "important_segments"
    }
}

processor_important = ClinicalProcessor(config_important)
result_important = processor_important.process_text(long_text)

tokenization = result_important.get('tokenization', {})
print("\nImportant Segments Strategy:")
print(f"  Text Length: {len(result_important['text'])} characters")
print(f"  Strategy: {tokenization.get('tokenization_strategy', 'N/A')}")

if 'num_segments_selected' in tokenization:
    selected = tokenization['num_segments_selected']
    total = tokenization['num_segments_total']
    percentage = (selected / total * 100) if total > 0 else 0
    print(f"  Segments: {selected}/{total} ({percentage:.1f}% selected)")

### 3.3 Custom Entity Recognition

In [ ]:
# Full entity recognition configuration
config_full_entities = {
    "entity_recognition": {
        "use_rules": True,
        "use_patterns": True,
        "cancer_specific_extraction": True,
        "temporal_extraction": True,
        "abbreviation_expansion": True,
        "ontologies": ["snomed_ct", "rxnorm"]
    }
}

processor_entities = ClinicalProcessor(config_full_entities)
result_entities = processor_entities.process_text(clinical_text)

print("\nFull Entity Recognition Configuration:")
print(f"  Entities Found: {len(result_entities['entities'])}")

# Count cancer-specific entities
cancer_types = ['tumor', 'staging', 'biomarker', 'measurement']
cancer_entities = [e for e in result_entities['entities'] if e['type'] in cancer_types]
print(f"  Cancer-Specific: {len(cancer_entities)}")

# Show sample cancer entities
print("\n  Sample Cancer Entities:")
for entity in cancer_entities[:5]:
    props = entity.get('properties', {})
    pattern = props.get('pattern', 'N/A')
    print(f"    • {entity['text']:25s} [{entity['type']:12s}] - {pattern}")

---
# 🤖 Section 4: Embedding Generation

Generate embeddings using biomedical language models.

### 4.1 Single Text Embedding

In [ ]:
# Sample texts for embedding
sample_text = "Patient diagnosed with stage IV lung adenocarcinoma."

print("Generating Embedding...")
print(f"Text: {sample_text}\n")

try:
    # Generate embedding with GatorTron
    embedding = processor.generate_embeddings(
        text=sample_text,
        model_name="gatortron",
        pooling_method="mean"
    )
    
    print("✓ Embedding Generated Successfully")
    print(f"\nEmbedding Details:")
    print(f"  Shape: {embedding.shape}")
    print(f"  Dimensions: {embedding.shape[1]}")
    print(f"  Mean: {np.mean(embedding):.4f}")
    print(f"  Std: {np.std(embedding):.4f}")
    print(f"  L2 Norm: {np.linalg.norm(embedding):.4f}")
    
except Exception as e:
    print(f"⚠ Embedding generation requires model download: {str(e)[:100]}")

### 4.2 Batch Embedding Generation

In [ ]:
# Multiple clinical texts
clinical_texts = [
    "Patient with breast cancer ER+ HER2-",
    "Lung adenocarcinoma stage IV with metastases",
    "Colorectal cancer with liver involvement",
    "Prostate cancer Gleason score 7"
]

print("Generating Batch Embeddings...")
print(f"Number of texts: {len(clinical_texts)}\n")

try:
    # Generate embeddings for all texts
    embeddings = processor.generate_embeddings(
        text=clinical_texts,
        model_name="gatortron",
        batch_size=2
    )
    
    print("✓ Batch Embeddings Generated")
    print(f"\nEmbeddings Shape: {embeddings.shape}")
    print(f"  - {embeddings.shape[0]} texts")
    print(f"  - {embeddings.shape[1]} dimensions each")
    
except Exception as e:
    print(f"⚠ {str(e)[:100]}")

### 4.3 Semantic Similarity

In [ ]:
try:
    # Compute pairwise similarities
    print("\nPairwise Cosine Similarities:")
    print("=" * 60)
    
    for i in range(len(clinical_texts)):
        for j in range(i + 1, len(clinical_texts)):
            sim = np.dot(embeddings[i], embeddings[j]) / (
                np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
            )
            print(f"Text {i+1} ↔ Text {j+1}: {sim:.4f}")
            
    # Find most similar pair
    max_sim = 0
    max_pair = (0, 0)
    
    for i in range(len(clinical_texts)):
        for j in range(i + 1, len(clinical_texts)):
            sim = np.dot(embeddings[i], embeddings[j]) / (
                np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
            )
            if sim > max_sim:
                max_sim = sim
                max_pair = (i, j)
    
    i, j = max_pair
    print(f"\n✓ Most Similar Texts (similarity: {max_sim:.4f}):")
    print(f"  1. {clinical_texts[i]}")
    print(f"  2. {clinical_texts[j]}")
    
except:
    print("Skipping similarity computation")

### 4.4 Different Pooling Methods

In [ ]:
pooling_methods = ["mean", "cls", "max"]
test_text = clinical_texts[0]

print(f"\nComparing Pooling Methods:")
print(f"Text: {test_text}")
print("=" * 60)

pooling_results = {}

for pooling in pooling_methods:
    try:
        emb = processor.generate_embeddings(
            text=test_text,
            model_name="gatortron",
            pooling_method=pooling
        )
        pooling_results[pooling] = emb
        
        print(f"\n{pooling.upper():12s}")
        print(f"  Shape: {emb.shape}")
        print(f"  Norm:  {np.linalg.norm(emb):.4f}")
        print(f"  Mean:  {np.mean(emb):.4f}")
        
    except Exception as e:
        print(f"\n{pooling.upper():12s} - Error: {str(e)[:50]}")

---
# 🧬 Section 5: Advanced Entity Extraction

Deep dive into entity extraction capabilities.

In [ ]:
# Comprehensive clinical text for entity extraction
comprehensive_text = """
PATHOLOGY REPORT

Patient: John Doe, Age 58
Date: March 20, 2024

FINAL DIAGNOSIS: Invasive Ductal Carcinoma, Left Breast, Stage IIA

CLINICAL HISTORY:
Patient presented on February 15, 2024 with a palpable 1.8 cm mass in the left
upper outer quadrant. Core biopsy performed on February 25, 2024 confirmed
invasive carcinoma.

GROSS DESCRIPTION:
Lumpectomy specimen measuring 5 x 4 x 3 cm. Sectioning reveals a 1.8 x 1.5 x 1.2 cm
firm, gray-white tumor.

MICROSCOPIC FINDINGS:
Invasive ductal carcinoma, Grade 2 (Nottingham score 6)
- Tubule formation: 20% (score 2)
- Nuclear pleomorphism: Moderate (score 2)
- Mitotic rate: 8 per 10 HPF (score 2)

IMMUNOHISTOCHEMISTRY:
ER: Positive (95%, strong)
PR: Positive (80%, moderate)
HER2: Negative (IHC 1+)
Ki-67: 18%

MOLECULAR:
EGFR: Wild-type
KRAS: Wild-type  
BRAF: No mutation
PD-L1: Positive (15%)

TNM: pT1c N0 M0, Stage IA

MARGINS: All negative (>2mm)

LYMPH NODES: 0/3 sentinel nodes positive

TREATMENT:
Started tamoxifen 20 mg PO daily on March 25, 2024
AC chemotherapy (doxorubicin 60 mg/m² + cyclophosphamide 600 mg/m²) initiated April 1, 2024
Radiation therapy planned for June 2024
Follow-up: May 1, 2024
"""

# Process with full entity recognition
config_comprehensive = {
    "entity_recognition": {
        "use_rules": True,
        "use_patterns": True,
        "cancer_specific_extraction": True,
        "temporal_extraction": True,
        "ontologies": ["snomed_ct", "rxnorm"]
    }
}

processor_comp = ClinicalProcessor(config_comprehensive)
result_comp = processor_comp.process_text(comprehensive_text)

print("Comprehensive Entity Extraction Results")
print("=" * 60)
print(f"Total Entities: {len(result_comp['entities'])}")
print(f"Relationships: {len(result_comp['entity_relationships'])}")
print(f"Timeline Events: {len(result_comp['temporal_timeline'])}")

In [ ]:
# Analyze cancer-specific entities
print("\nCancer-Specific Entities by Type:")
print("=" * 60)

cancer_types = ['tumor', 'staging', 'biomarker', 'measurement']

for cancer_type in cancer_types:
    entities = [e for e in result_comp['entities'] if e['type'] == cancer_type]
    
    if entities:
        print(f"\n{cancer_type.upper()} ({len(entities)} found):")
        for entity in entities[:5]:  # Show first 5
            text = entity['text']
            props = entity.get('properties', {})
            pattern = props.get('pattern', 'N/A')
            print(f"  • {text:30s} (pattern: {pattern})")

In [ ]:
# Display biomarker information
biomarkers = [e for e in result_comp['entities'] if e['type'] == 'biomarker']

if biomarkers:
    print("\nBiomarker Status:")
    print("=" * 60)
    
    for biomarker in biomarkers:
        text = biomarker['text']
        props = biomarker.get('properties', {})
        
        print(f"\n{text}")
        if props.get('biomarker'):
            print(f"  Marker: {props['biomarker']}")
        if props.get('status'):
            print(f"  Status: {props['status']}")

In [ ]:
# Get summary statistics
stats = processor_comp.get_summary_statistics(result_comp)

print("\nSummary Statistics:")
print("=" * 60)
print(f"Text Length: {stats['text_length']} characters")
print(f"Total Entities: {stats['num_entities']}")
print(f"Relationships: {stats['num_relationships']}")
print(f"Timeline Events: {stats['num_timeline_events']}")

print("\nEntity Distribution:")
for entity_type, count in sorted(stats['entity_types'].items()):
    percentage = (count / stats['num_entities'] * 100) if stats['num_entities'] > 0 else 0
    bar = "█" * min(int(percentage / 2), 50)
    print(f"  {entity_type:15s} {count:3d} ({percentage:5.1f}%) {bar}")

---
# 🔗 Section 6: HoneyBee Main API

Use the high-level HoneyBee API for clinical processing.

In [ ]:
# Initialize HoneyBee with custom configuration
honeybee_config = {
    "clinical": {
        "tokenization": {
            "model": "gatortron"
        },
        "entity_recognition": {
            "cancer_specific_extraction": True,
            "temporal_extraction": True
        }
    }
}

honeybee = HoneyBee(config=honeybee_config)

print("✓ HoneyBee Initialized")
print(f"Configuration: Custom clinical settings")

### 6.1 Process Clinical Text

In [ ]:
# Process text using HoneyBee API
sample_clinical = "Patient diagnosed with stage III breast cancer, ER+/PR+/HER2-, started on tamoxifen therapy."

result_hb = honeybee.process_clinical(text=sample_clinical)

print("HoneyBee API - Process Clinical Text")
print("=" * 60)
print(f"\nInput: {sample_clinical}")
print(f"\nResults:")
print(f"  Entities Found: {len(result_hb['entities'])}")

if result_hb['entities']:
    print(f"\n  Extracted Entities:")
    for entity in result_hb['entities']:
        print(f"    • {entity['text']:30s} [{entity['type']}]")

### 6.2 Generate Embeddings

In [ ]:
try:
    # Single text embedding
    embedding_single = honeybee.generate_embeddings(
        data=sample_clinical,
        modality="clinical",
        model_name="gatortron"
    )
    
    print("\nHoneyBee API - Generate Embeddings")
    print("=" * 60)
    print(f"Single text embedding: {embedding_single.shape}")
    
    # Batch embeddings
    texts = [
        "Patient with lung cancer",
        "Started chemotherapy treatment",
        "Follow-up shows partial response"
    ]
    
    embeddings_batch = honeybee.generate_embeddings(
        data=texts,
        modality="clinical",
        model_name="gatortron"
    )
    
    print(f"Batch embeddings: {embeddings_batch.shape}")
    print(f"  - {embeddings_batch.shape[0]} texts")
    print(f"  - {embeddings_batch.shape[1]} dimensions")
    
except Exception as e:
    print(f"⚠ Embeddings: {str(e)[:100]}")

### 6.3 Multimodal Integration

In [ ]:
try:
    # Generate embeddings from different sources
    emb1 = honeybee.generate_embeddings(
        data="Patient with breast cancer",
        modality="clinical"
    )
    
    emb2 = honeybee.generate_embeddings(
        data="ER positive HER2 negative",
        modality="clinical"
    )
    
    # Integrate embeddings
    integrated = honeybee.integrate_embeddings([emb1, emb2])
    
    print("\nMultimodal Integration:")
    print("=" * 60)
    print(f"Embedding 1: {emb1.shape}")
    print(f"Embedding 2: {emb2.shape}")
    print(f"Integrated:  {integrated.shape}")
    print(f"\n✓ Successfully integrated {len([emb1, emb2])} embeddings")
    
except Exception as e:
    print(f"⚠ Integration: {str(e)[:100]}")

---
# 📦 Section 7: Batch Processing

Process multiple documents efficiently.

In [ ]:
# Note: This example uses processor.process_batch()
# For demonstration, we'll show how it would be used

print("Batch Processing Example")
print("=" * 60)
print("""
To process multiple documents:

```python
# Process all PDFs in a directory
results = processor.process_batch(
    input_dir="./clinical_reports",
    file_pattern="*.pdf",
    save_output=True
)

# Analyze results
for result in results:
    if 'error' not in result:
        print(f"File: {result['file_name']}")
        print(f"  Entities: {len(result['entities'])}")
        print(f"  Timeline Events: {len(result['temporal_timeline'])}")
```

Features:
- Automatic file discovery with glob patterns
- Parallel processing support
- JSON output generation
- Error handling per file
- Progress tracking
""")

---
# 🎯 Section 8: Complete Workflow Examples

End-to-end examples for common use cases.

### 8.1 Cancer Patient Analysis Pipeline

In [ ]:
def analyze_cancer_patient(text_or_path):
    """
    Complete pipeline for cancer patient analysis
    """
    # Initialize with full configuration
    config = {
        "entity_recognition": {
            "cancer_specific_extraction": True,
            "temporal_extraction": True,
            "use_rules": True,
            "use_patterns": True
        }
    }
    
    processor = ClinicalProcessor(config)
    
    # Process input
    if Path(text_or_path).exists():
        result = processor.process(text_or_path)
    else:
        result = processor.process_text(text_or_path)
    
    # Extract key information
    analysis = {
        'tumor_entities': [e for e in result['entities'] if e['type'] == 'tumor'],
        'staging': [e for e in result['entities'] if e['type'] == 'staging'],
        'biomarkers': [e for e in result['entities'] if e['type'] == 'biomarker'],
        'timeline': result['temporal_timeline'],
        'total_entities': len(result['entities'])
    }
    
    return analysis

# Example usage
example_text = """
Patient with invasive ductal carcinoma, ER+/PR+/HER2-, Grade 2, Stage IIA.
Tumor size 2.3 cm. Started on tamoxifen 20mg daily.
"""

analysis = analyze_cancer_patient(example_text)

print("Cancer Patient Analysis")
print("=" * 60)
print(f"Total Entities: {analysis['total_entities']}")
print(f"Tumor Mentions: {len(analysis['tumor_entities'])}")
print(f"Staging Info: {len(analysis['staging'])}")
print(f"Biomarkers: {len(analysis['biomarkers'])}")
print(f"Timeline Events: {len(analysis['timeline'])}")

### 8.2 Similarity Search Pipeline

In [ ]:
def find_similar_cases(query_text, case_database, top_k=3):
    """
    Find similar clinical cases using embeddings
    """
    try:
        processor = ClinicalProcessor()
        
        # Generate query embedding
        query_emb = processor.generate_embeddings(
            text=query_text,
            model_name="gatortron"
        )
        
        # Generate database embeddings
        db_embs = processor.generate_embeddings(
            text=case_database,
            model_name="gatortron"
        )
        
        # Compute similarities
        similarities = []
        for i, db_emb in enumerate(db_embs):
            sim = np.dot(query_emb.flatten(), db_emb) / (
                np.linalg.norm(query_emb) * np.linalg.norm(db_emb)
            )
            similarities.append((i, sim))
        
        # Sort and get top-k
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        return similarities[:top_k]
        
    except Exception as e:
        print(f"Error: {str(e)[:100]}")
        return []

# Example database
case_db = [
    "Breast cancer ER+ HER2- stage II",
    "Lung adenocarcinoma EGFR mutant",
    "Invasive ductal carcinoma grade 2",
    "Colorectal cancer with liver mets"
]

query = "Breast carcinoma hormone receptor positive"

print("Similarity Search Example")
print("=" * 60)
print(f"Query: {query}\n")

similar = find_similar_cases(query, case_db, top_k=3)

if similar:
    print("Top Similar Cases:")
    for idx, (case_idx, similarity) in enumerate(similar, 1):
        print(f"{idx}. {case_db[case_idx]}")
        print(f"   Similarity: {similarity:.4f}\n")

---
# 📊 Summary and Best Practices

## Key Capabilities

✅ **Document Processing**
- PDF, images with OCR
- Direct text processing
- Batch processing
- EHR format support

✅ **Entity Extraction**
- Cancer-specific entities
- Temporal information
- Entity relationships
- Ontology mapping

✅ **Embeddings**
- Multiple biomedical models
- Different pooling methods
- Batch processing
- Similarity computation

✅ **Configuration**
- Flexible tokenization
- Long document strategies
- Custom entity recognition
- Output customization

## Best Practices

1. **Choose the right model**
   - GatorTron: Clinical text (1024 dims)
   - BioClinicalBERT: Clinical reports (768 dims)
   - PubMedBERT: Research literature (768 dims)

2. **Configure appropriately**
   - Use `important_segments` for long documents
   - Enable `cancer_specific_extraction` for oncology
   - Enable `temporal_extraction` for timelines

3. **Optimize performance**
   - Use batch processing for multiple texts
   - Configure appropriate batch sizes
   - Cache embeddings when possible

4. **Handle errors gracefully**
   - Check for OCR quality
   - Validate entity extraction
   - Monitor processing times

## Resources

- **Documentation**: https://lab-rasool.github.io/HoneyBee/docs/clinical-processing/
- **GitHub**: https://github.com/lab-rasool/HoneyBee
- **Issues**: https://github.com/lab-rasool/HoneyBee/issues

---

**End of Notebook** 🎉